# Bridge Infrastructure Data Exploration

This notebook explores the bridge infrastructure data to understand:
- Data quality and completeness
- Distribution of bridge characteristics
- Geographic patterns
- Condition trends and correlations

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')

# Configuration
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## 1. Data Loading and Initial Inspection

In [ ]:
# Load bridge data
data_path = Path('../data/raw/bridges')

# Load inventory data
inventory_df = pd.read_csv(data_path / 'bridge_inventory.csv')
print(f"Bridge inventory: {inventory_df.shape[0]} rows, {inventory_df.shape[1]} columns")

# Load conditions data
conditions_df = pd.read_csv(data_path / 'bridge_conditions.csv')
print(f"Bridge conditions: {conditions_df.shape[0]} rows, {conditions_df.shape[1]} columns")

# Display first few rows
print("\nBridge Inventory Data:")
display(inventory_df.head())

print("\nBridge Conditions Data:")
display(conditions_df.head())

In [ ]:
# Data info and missing values
print("=== BRIDGE INVENTORY DATA INFO ===")
print(inventory_df.info())
print("\nMissing Values:")
print(inventory_df.isnull().sum())

print("\n\n=== BRIDGE CONDITIONS DATA INFO ===")
print(conditions_df.info())
print("\nMissing Values:")
print(conditions_df.isnull().sum())

## 2. Data Quality Assessment

In [ ]:
# Merge datasets
merged_df = pd.merge(inventory_df, conditions_df, on='bridge_id', how='inner')
print(f"Merged dataset: {merged_df.shape[0]} rows, {merged_df.shape[1]} columns")

# Basic statistics
print("\n=== BASIC STATISTICS ===")
display(merged_df.describe())

## 3. Bridge Characteristics Analysis

In [ ]:
# Create visualizations
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Age distribution
current_year = 2023
merged_df['age'] = current_year - merged_df['construction_year']
axes[0, 0].hist(merged_df['age'], bins=15, alpha=0.7, edgecolor='black')
axes[0, 0].set_title('Bridge Age Distribution')
axes[0, 0].set_xlabel('Age (years)')
axes[0, 0].set_ylabel('Count')

# Length distribution
axes[0, 1].hist(merged_df['length_m'], bins=15, alpha=0.7, edgecolor='black', color='orange')
axes[0, 1].set_title('Bridge Length Distribution')
axes[0, 1].set_xlabel('Length (meters)')
axes[0, 1].set_ylabel('Count')

# Material distribution
material_counts = merged_df['material'].value_counts()
axes[0, 2].pie(material_counts.values, labels=material_counts.index, autopct='%1.1f%%')
axes[0, 2].set_title('Bridge Material Distribution')

# Condition score distribution
axes[1, 0].hist(merged_df['condition_score'], bins=15, alpha=0.7, edgecolor='black', color='green')
axes[1, 0].set_title('Condition Score Distribution')
axes[1, 0].set_xlabel('Condition Score')
axes[1, 0].set_ylabel('Count')

# Overall rating distribution
axes[1, 1].hist(merged_df['overall_rating'], bins=15, alpha=0.7, edgecolor='black', color='red')
axes[1, 1].set_title('Overall Rating Distribution')
axes[1, 1].set_xlabel('Overall Rating')
axes[1, 1].set_ylabel('Count')

# Maintenance needed
maint_counts = merged_df['maintenance_needed'].value_counts()
axes[1, 2].bar(maint_counts.index.astype(str), maint_counts.values, alpha=0.7)
axes[1, 2].set_title('Maintenance Needed')
axes[1, 2].set_xlabel('Maintenance Needed')
axes[1, 2].set_ylabel('Count')

plt.tight_layout()
plt.show()

## 4. Geographic Distribution Analysis

In [ ]:
# Create a map of bridge locations
center_lat = merged_df['latitude'].mean()
center_lon = merged_df['longitude'].mean()

# Create base map
m = folium.Map(location=[center_lat, center_lon], zoom_start=11)

# Add bridges to map with color coding by condition
def get_color(condition_score):
    if condition_score >= 8.0:
        return 'green'
    elif condition_score >= 7.0:
        return 'yellow'
    elif condition_score >= 6.0:
        return 'orange'
    else:
        return 'red'

for idx, row in merged_df.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=8,
        popup=f"{row['name']}\nCondition: {row['condition_score']}",
        color=get_color(row['condition_score']),
        fill=True,
        opacity=0.7
    ).add_to(m)

# Add legend
legend_html = '''
<div style="position: fixed; 
            top: 10px; right: 10px; width: 150px; height: 120px; 
            background-color: white; border:2px solid grey; z-index:9999; 
            font-size:14px; padding: 10px">
<h4>Bridge Condition</h4>
<i class="fa fa-circle" style="color:green"></i> Excellent (8.0+)<br>
<i class="fa fa-circle" style="color:yellow"></i> Good (7.0-7.9)<br>
<i class="fa fa-circle" style="color:orange"></i> Fair (6.0-6.9)<br>
<i class="fa fa-circle" style="color:red"></i> Poor (<6.0)
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Display map
m

## 5. Correlation Analysis

In [ ]:
# Select numeric columns for correlation analysis
numeric_cols = ['construction_year', 'length_m', 'width_m', 'condition_score', 
                'overall_rating', 'age']

# Create correlation matrix
correlation_matrix = merged_df[numeric_cols].corr()

# Plot correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, fmt='.2f', cbar_kws={'shrink': 0.8})
plt.title('Bridge Characteristics Correlation Matrix')
plt.tight_layout()
plt.show()

print("\n=== KEY CORRELATIONS ===")
# Find highest correlations with condition score
condition_corrs = correlation_matrix['condition_score'].abs().sort_values(ascending=False)
print("Correlations with Condition Score:")
for var, corr in condition_corrs.items():
    if var != 'condition_score':
        print(f"{var}: {corr:.3f}")

## 6. Condition Analysis by Categories

In [ ]:
# Analyze condition by material type
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Condition by material
sns.boxplot(data=merged_df, x='material', y='condition_score', ax=axes[0])
axes[0].set_title('Condition Score by Material Type')
axes[0].tick_params(axis='x', rotation=45)

# Condition by age groups
merged_df['age_group'] = pd.cut(merged_df['age'], 
                               bins=[0, 15, 30, 45, 100], 
                               labels=['0-15', '16-30', '31-45', '45+'])
sns.boxplot(data=merged_df, x='age_group', y='condition_score', ax=axes[1])
axes[1].set_title('Condition Score by Age Group')

# Condition by length categories
merged_df['length_category'] = pd.cut(merged_df['length_m'], 
                                     bins=[0, 50, 100, 150, 1000], 
                                     labels=['Short', 'Medium', 'Long', 'Very Long'])
sns.boxplot(data=merged_df, x='length_category', y='condition_score', ax=axes[2])
axes[2].set_title('Condition Score by Length Category')

plt.tight_layout()
plt.show()

## 7. Summary Statistics and Insights

In [ ]:
print("=== BRIDGE DATA SUMMARY ===")
print(f"Total bridges analyzed: {len(merged_df)}")
print(f"Average age: {merged_df['age'].mean():.1f} years")
print(f"Average condition score: {merged_df['condition_score'].mean():.2f}")
print(f"Bridges needing maintenance: {merged_df['maintenance_needed'].sum()}")
print(f"Percentage needing maintenance: {(merged_df['maintenance_needed'].sum() / len(merged_df)) * 100:.1f}%")

print("\n=== CONDITION DISTRIBUTION ===")
condition_bins = pd.cut(merged_df['condition_score'], 
                       bins=[0, 5, 6, 7, 8, 10], 
                       labels=['Poor', 'Fair', 'Good', 'Very Good', 'Excellent'])
condition_dist = condition_bins.value_counts().sort_index()
for condition, count in condition_dist.items():
    percentage = (count / len(merged_df)) * 100
    print(f"{condition}: {count} bridges ({percentage:.1f}%)")

print("\n=== MATERIAL ANALYSIS ===")
material_stats = merged_df.groupby('material').agg({
    'condition_score': ['count', 'mean', 'std'],
    'age': 'mean',
    'maintenance_needed': 'sum'
}).round(2)
print(material_stats)

print("\n=== KEY INSIGHTS ===")
print("1. Age-Condition Relationship:")
age_condition_corr = merged_df['age'].corr(merged_df['condition_score'])
print(f"   - Correlation between age and condition: {age_condition_corr:.3f}")

print("\n2. Material Performance:")
best_material = merged_df.groupby('material')['condition_score'].mean().idxmax()
best_score = merged_df.groupby('material')['condition_score'].mean().max()
print(f"   - Best performing material: {best_material} (avg score: {best_score:.2f})")

print("\n3. Maintenance Priority:")
urgent_bridges = merged_df[merged_df['condition_score'] < 6.0]
print(f"   - Bridges requiring urgent attention: {len(urgent_bridges)}")
if len(urgent_bridges) > 0:
    print(f"   - Average age of urgent bridges: {urgent_bridges['age'].mean():.1f} years")